<a href="https://colab.research.google.com/github/shAnant/twitter-sentiment-analysis/blob/main/colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
training_data = pd.read_csv("/content/twitter_training.csv")
validation_data = pd.read_csv("/content/twitter_validation.csv")

In [ ]:
training_data.head()

,number,user,sentiment,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
training_data.shape

(74681, 4)

In [ ]:
testing_data = training_data.tail(100)
training_data = training_data.iloc[:-100]

In [ ]:
training_data.shape

(74581, 4)

In [ ]:
data = training_data.drop(['number','user'],axis = 1)

In [ ]:
data.head()

,sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [ ]:
data.isnull().sum()

,0
sentiment,0
text,686


In [ ]:
data.dropna(subset=['text'],inplace = True)

In [ ]:
data = data.sample(frac = 1)

In [ ]:
data.head()

,sentiment,text
46412,Negative,Anybody in the southeastern PA area reporting ...
53313,Positive,Red Dead or Redemption is pretty dope
67813,Negative,About the Shit Son
36008,Positive,Solid shout here from @The_Ashes2Ashes - Commu...
31400,Negative,League of Legends is having issues since 1:35 ...


In [ ]:
data.isnull().sum()

,0
sentiment,0
text,0


In [ ]:
import re
import string
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_tweet(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub('\[.*?]','',text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<.*?>+',b'',text)
    text = re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text = re.sub('\n','',text)
    text = re.sub('\w*\d\w*','',text)
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data['text'] = data['text'].apply(preprocess_tweet)

In [ ]:
x = data['text']
y = data['sentiment']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer(ngram_range=(1,2),stop_words='english',max_df=0.9,min_df=5)
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
# LR = LogisticRegression(C=10)
# LR.fit(xv_train,y_train)

# from sklearn.svm import SVC
# svm_model = SVC(C=1.0, kernel='linear')  # Using linear kernel
# svm_model.fit(xv_train, y_train)

from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(xv_train, y_train)


MultinomialNB()

In [ ]:
pred_lr = nb_model.predict(xv_test)

In [ ]:
nb_model.score(xv_test,y_test)

0.7197683230486088

In [ ]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

  Irrelevant       0.83      0.75      0.79      3287
    Negative       0.86      0.86      0.86      5647
     Neutral       0.83      0.80      0.82      4489
    Positive       0.78      0.86      0.82      5051

    accuracy                           0.82     18474
   macro avg       0.83      0.82      0.82     18474
weighted avg       0.83      0.82      0.82     18474



In [ ]:
import pickle

# Assuming your trained model is stored in the variable 'LR'
# and your vectorizer is stored in 'vectorization'

filename = 'finalized_model.sav'
pickle.dump(LR, open(filename, 'wb'))

filename = 'vectorizer.sav'
pickle.dump(vectorization, open(filename, 'wb'))